In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from skl2onnx.helpers.onnx_helper import load_onnx_model
from stitchnet.stitchonnx.utils import Net
from glob import glob
from pathlib import Path

netsFiles = sorted(glob('_models/fragments/net*'))
nets = []
for i,netsFile in enumerate(netsFiles):
    fragmentFiles = sorted(glob(str(Path(netsFile)/'fragment*.onnx')))
    onnxFragments = []
    for fragmentFile in fragmentFiles:
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
    net1 = Net(onnxFragments, i)
    nets.append(net1)

In [ ]:
from stitchnet.stitchonnx.utils import load_cats_and_dogs_dset,convert_imagenet_to_cat_dog_label
from stitchnet.stitchonnx.utils import accuracy_score_model,accuracy_score_net,load_dl
from stitchnet.stitchonnx.utils import generate_networks, ScoreMapper
from stitchnet.stitchonnx.report import Report
from stitchnet.stitchonnx.utils import evalulate_stitchnet

from tqdm import tqdm
import torch
import numpy as np
import os
from collections import defaultdict
import hashlib

STITCH_BATCH_SIZE = 64
EVAL_BATCH_SIZE = 64
MAX_DEPTH = 10
THRESOULD = 0.8
TOTAL_THRESOULD = 0.5

RESULT_NAME = "result6"

dataset_train = load_cats_and_dogs_dset("train")
dataset_val = load_cats_and_dogs_dset("test")

dl_score = load_dl(dataset_train, STITCH_BATCH_SIZE)
data_score,t = next(iter(dl_score))
data_score = data_score.numpy()

In [ ]:
# range(1)
k = 0
if os.path.exists(f'./_results/{RESULT_NAME}.txt'):
    with open(f'./_results/{RESULT_NAME}.txt', 'r') as f:
        k = len(f.read().split('\n'))

In [ ]:
from tqdm import tqdm

scoreMapper = ScoreMapper(nets, data_score)
with Report(EVAL_BATCH_SIZE, f'./_results/{RESULT_NAME}.txt', 'a') as report:
    for _ in tqdm(range(50)):
        generator = generate_networks(nets, scoreMapper, data_score, 
                              threshold=THRESOULD, totalThreshold=TOTAL_THRESOULD, 
                              maxDepth=MAX_DEPTH, sample=True)
        for i,(s,net) in enumerate(generator):
            netname = f"_results/{RESULT_NAME}/net{k:03}_{i:03}"
            report.evaluate(nets, net, netname, s, dataset_val)
            k += 1